In [35]:
import pandas as pd
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing
from aif360.metrics import ClassificationMetric
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np
from itertools import combinations

In [2]:
data = pd.read_csv('cleaned_tripdata_ACSData_2023-09.csv')

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
data.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag,pickup_zipcode,dropoff_zipcode,zipcode,Households!!Total,Households!!Median income (dollars),Households!!Mean income (dollars),Families!!Total,Total!!Population for whom poverty status is determined,Total!!Population for whom poverty status is determined!!SEX!!Male,Total!!Population for whom poverty status is determined!!SEX!!Female,Below poverty level!!Population for whom poverty status is determined,Percent below poverty level!!Population for whom poverty status is determined,Total number of housing units\n,Total number of Occupied housing units,Total number of vacant housing units,Total population,RACE!!Total population!!One race!!White,RACE!!Total population!!One race!!Black or African American,RACE!!Total population!!One race!!American Indian and Alaska Native,RACE!!Total population!!One race!!Asian,RACE!!Total population!!One race!!Some Other Race
0,HV0003,B03404,B03404,2023-09-01 00:00:13,2023-09-01 00:02:47,2023-09-01 00:02:56,2023-09-01 00:14:08,7,226,2.35,672,11.57,0.0,0.32,1.03,0.00,0.0,0.00,9.41,N,N,,N,N,11106.0,11104.0,11104.0,12234.0,84716,106656,6252.0,26744.0,12972.0,13772.0,2348.0,8.8,13224.0,12234.0,990.0,26825.0,11994.0,779.0,54.0,7601.0,2228.0
1,HV0003,B03404,B03404,2023-09-01 00:27:24,2023-09-01 00:29:32,2023-09-01 00:29:44,2023-09-01 00:59:58,144,36,6.79,1814,29.72,0.0,0.82,2.64,2.75,0.0,7.18,25.97,N,N,,N,N,10012.0,11237.0,11237.0,17723.0,82570,99019,8064.0,45077.0,23061.0,22016.0,9391.0,20.8,18801.0,17723.0,1078.0,45233.0,14395.0,5924.0,1100.0,3084.0,13170.0
2,HV0003,B03404,B03404,2023-09-01 00:25:14,2023-09-01 00:27:15,2023-09-01 00:28:01,2023-09-01 00:35:53,186,125,1.61,472,10.89,0.0,0.30,0.97,2.75,0.0,0.00,8.34,N,N,,N,N,10001.0,10013.0,10013.0,12609.0,159474,385678,6489.0,26287.0,12439.0,13848.0,2609.0,9.9,15933.0,12609.0,3324.0,28215.0,15873.0,1053.0,14.0,8621.0,680.0
3,HV0003,B03404,B03404,2023-09-01 00:31:44,2023-09-01 00:36:51,2023-09-01 00:36:51,2023-09-01 00:46:48,125,148,1.37,597,9.60,0.0,0.26,0.85,0.75,0.0,0.00,7.44,Y,N,,N,N,10013.0,10002.0,10002.0,35771.0,46525,93314,16924.0,74729.0,37794.0,36935.0,18611.0,24.9,39646.0,35771.0,3875.0,75517.0,23597.0,7074.0,262.0,27700.0,9006.0
4,HV0003,B03404,B03404,2023-09-01 00:08:33,2023-09-01 00:10:53,2023-09-01 00:12:28,2023-09-01 00:27:47,170,48,1.29,919,16.29,0.0,0.45,1.45,2.75,0.0,0.00,13.30,N,N,,N,Y,10016.0,10019.0,10019.0,26900.0,121835,198898,7667.0,43950.0,24087.0,19863.0,4682.0,10.7,33279.0,26900.0,6379.0,44276.0,26883.0,2740.0,239.0,8151.0,1976.0


## Preprocessing

In [5]:
data['Households!!Median income (dollars)'] = pd.to_numeric(data['Households!!Median income (dollars)'], errors='coerce')
threshold = data['Households!!Median income (dollars)'].quantile(0.25)
data['income_level'] = (data['Households!!Median income (dollars)'] > threshold).astype(int)    #0: low; 1: high

In [6]:
data['Percent below poverty level!!Population for whom poverty status is determined'] = pd.to_numeric(data['Percent below poverty level!!Population for whom poverty status is determined'], errors='coerce')
poverty_threshold = data['Percent below poverty level!!Population for whom poverty status is determined'].median()
data['poverty_status'] = (data['Percent below poverty level!!Population for whom poverty status is determined'] <= poverty_threshold).astype(int)     #0: high; 1: low

In [7]:
race_columns = [
    'RACE!!Total population!!One race!!White',
    'RACE!!Total population!!One race!!Black or African American',
    'RACE!!Total population!!One race!!American Indian and Alaska Native',
    'RACE!!Total population!!One race!!Asian',
    'RACE!!Total population!!One race!!Some Other Race'
]

data['majority_race'] = data[race_columns].idxmax(axis=1)
data['race'] = data['majority_race'].apply(
    lambda x: 0 if x == 'RACE!!Total population!!One race!!Black or African American' else 1
)

In [8]:
data = data[['base_passenger_fare', 'trip_miles', 'trip_time', 'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag', 'wav_request_flag', 'wav_match_flag', 'income_level', 'poverty_status', 'race']]

In [9]:
fare_threshold = data['base_passenger_fare'].quantile(0.75)
data['fare_level'] = (data['base_passenger_fare'] > fare_threshold).astype(int)
data = data.drop('base_passenger_fare', axis=1)

In [10]:
for col in ['shared_request_flag', 'shared_match_flag',
            'access_a_ride_flag', 'wav_request_flag', 'wav_match_flag']:
    data[col] = (data[col] == 'Y').astype(int)

In [11]:
data.head()

,trip_miles,trip_time,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag,income_level,poverty_status,race,fare_level
0,2.35,672,0,0,0,0,0,1,1,1,0
1,6.79,1814,0,0,0,0,0,1,0,1,1
2,1.61,472,0,0,0,0,0,1,1,1,0
3,1.37,597,1,0,0,0,0,0,0,1,0
4,1.29,919,0,0,0,0,1,1,1,1,0


## Helper Functions

In [ ]:
label_O = 'race'    # protected attribute
label_Y = 'fare_level'  # prediction target

In [ ]:
def evaluate_crossdf_metrics_binary(cross_df):
    metrics = {}
    group0 = cross_df[cross_df['label_O'] == 0]
    group1 = cross_df[cross_df['label_O'] == 1]
    def safe_rate(num, denom):
        return num / denom if denom > 0 else 0

    pr0 = (group0['pred_Y'] == 1).mean()
    pr1 = (group1['pred_Y'] == 1).mean()
    metrics['delta_sp'] = abs(pr0 - pr1)

    group0_pos = group0[group0['label_Y'] == 1]
    group1_pos = group1[group1['label_Y'] == 1]

    tpr0 = safe_rate((group0_pos['pred_Y'] == 1).sum(), len(group0_pos))
    tpr1 = safe_rate((group1_pos['pred_Y'] == 1).sum(), len(group1_pos))
    fnr0 = safe_rate((group0_pos['pred_Y'] == 0).sum(), len(group0_pos))
    fnr1 = safe_rate((group1_pos['pred_Y'] == 0).sum(), len(group1_pos))

    metrics['delta_eopp'] = abs(tpr0 - tpr1)
    metrics['delta_fnrb'] = abs(fnr0 - fnr1)

    group0_neg = group0[group0['label_Y'] == 0]
    group1_neg = group1[group1['label_Y'] == 0]

    fpr0 = safe_rate((group0_neg['pred_Y'] == 1).sum(), len(group0_neg))
    fpr1 = safe_rate((group1_neg['pred_Y'] == 1).sum(), len(group1_neg))

    metrics['delta_fprb'] = abs(fpr0 - fpr1)


    def calc_ppv(temp):
        TP = ((temp['label_Y'] == 1) & (temp['pred_Y'] == 1)).sum()
        FP = ((temp['label_Y'] == 0) & (temp['pred_Y'] == 1)).sum()
        return safe_rate(TP, TP + FP)

    def calc_npv(temp):
        TN = ((temp['label_Y'] == 0) & (temp['pred_Y'] == 0)).sum()
        FN = ((temp['label_Y'] == 1) & (temp['pred_Y'] == 0)).sum()
        return safe_rate(TN, TN + FN)

    ppv0 = calc_ppv(group0)
    ppv1 = calc_ppv(group1)
    npv0 = calc_npv(group0)
    npv1 = calc_npv(group1)

    metrics['delta_ppvp'] = abs(ppv0 - ppv1)
    metrics['delta_npvp'] = abs(npv0 - npv1)


    acc0 = (group0['pred_Y'] == group0['label_Y']).mean()
    acc1 = (group1['pred_Y'] == group1['label_Y']).mean()
    metrics['delta_oae'] = abs(acc0 - acc1)

    if 'pred_prob' in cross_df.columns:
        predprob_pos0 = group0[group0['label_Y'] == 1]['pred_prob'].mean()
        predprob_pos1 = group1[group1['label_Y'] == 1]['pred_prob'].mean()
        predprob_neg0 = group0[group0['label_Y'] == 0]['pred_prob'].mean()
        predprob_neg1 = group1[group1['label_Y'] == 0]['pred_prob'].mean()

        metrics['delta_bpc'] = abs(predprob_pos0 - predprob_pos1)
        metrics['delta_bnc'] = abs(predprob_neg0 - predprob_neg1)
    else:
        metrics['delta_bpc'] = np.nan
        metrics['delta_bnc'] = np.nan

    metrics['delta_cuae'] = abs(ppv0 - ppv1) + abs(npv0 - npv1)
    metrics['delta_eo'] = (abs(tpr0 - tpr1) + abs(fpr0 - fpr1)) / 2

    return metrics

# Reweighing

In [34]:
# Step 1: Split the raw data into train/test

dataset = BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=data,
    label_names=[label_Y],
    protected_attribute_names=[label_O]
)

data_train, data_test = train_test_split(data, test_size=0.2, random_state=42, stratify=data[label_Y])

dataset_train = BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=data_train,
    label_names=[label_Y],
    protected_attribute_names=[label_O]
)

dataset_test = BinaryLabelDataset(
    favorable_label=1,
    unfavorable_label=0,
    df=data_test,
    label_names=[label_Y],
    protected_attribute_names=[label_O]
)

In [37]:
# Step 2: Train Logistic Regression (Baseline, no mitigation)
X_train = data_train.drop(columns=[label_Y])
y_train = data_train[label_Y]

X_test = data_test.drop(columns=[label_Y])
y_test = data_test[label_Y]

X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_test = X_test.apply(pd.to_numeric, errors='coerce')

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_prob = clf.predict_proba(X_test)[:, 1]

In [46]:
# Step 3: Create cross_df_base
cross_df_base = pd.DataFrame({
    'label_O': data_test[label_O].values,  # protected attribute
    'label_Y': y_test.values,              # true label
    'pred_Y': y_pred,                      # predicted label
    'pred_prob': y_prob                    # predicted probability for positive class
})

In [ ]:
# Step 4: Apply Reweighing to training dataset
RW = Reweighing(
    privileged_groups=[{label_O: 1}],
    unprivileged_groups=[{label_O: 0}]
)

dataset_train_rw = RW.fit_transform(dataset_train)

In [43]:
# Step 6: Train Logistic Regression after Reweighing
X_train_rw = dataset_train_rw.features
y_train_rw = dataset_train_rw.labels.ravel()
sample_weights_rw = dataset_train_rw.instance_weights

clf_rw = LogisticRegression(max_iter=1000)
clf_rw.fit(X_train_rw, y_train_rw, sample_weight=sample_weights_rw)

LogisticRegression(max_iter=1000)

In [44]:
y_pred_rw = clf_rw.predict(X_test)
y_prob_rw = clf_rw.predict_proba(X_test)[:, 1]

# Step 7: Create cross_df_rw
cross_df_rw = pd.DataFrame({
    'label_O': data_test[label_O].values,
    'label_Y': y_test.values,
    'pred_Y': y_pred_rw,
    'pred_prob': y_prob_rw
})

C:\Users\Jieyi Zhang\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
C:\Users\Jieyi Zhang\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [47]:
bias_base = evaluate_crossdf_metrics_binary(cross_df_base)
acc_base = (cross_df_base['pred_Y'] == cross_df_base['label_Y']).mean()

bias_rw = evaluate_crossdf_metrics_binary(cross_df_rw)
acc_rw = (cross_df_rw['pred_Y'] == cross_df_rw['label_Y']).mean()